In [13]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization, merge, Convolution2D
from keras import backend as K
from keras.datasets import mnist
from keras.models import Model

from matplotlib import pyplot as plt

import itertools

from sklearn import svm, metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = keras.utils.to_categorical(y_train, num_classes=10)
Y_test = keras.utils.to_categorical(y_test, num_classes=10)

AttributeError: 'Datasets' object has no attribute 'load_data'

In [3]:
# ResNet from https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32
total_layers = 25 #Specify how deep we want our network
units_between_stride = int(total_layers / 5)

def resUnit(input_layer,i):
    with tf.variable_scope("res_unit"+str(i)):
        part1 = slim.batch_norm(input_layer,activation_fn=None)
        part2 = tf.nn.relu(part1)
        part3 = slim.conv2d(part2,64,[3,3],activation_fn=None)
        part4 = slim.batch_norm(part3,activation_fn=None)
        part5 = tf.nn.relu(part4)
        part6 = slim.conv2d(part5,64,[3,3],activation_fn=None)
        output = input_layer + part6
        return output

tf.reset_default_graph()

input_layer = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32,name='input')
label_layer = tf.placeholder(shape=[None],dtype=tf.int32)
label_oh = slim.layers.one_hot_encoding(label_layer,10)

layer1 = slim.conv2d(input_layer,64,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(0))
for i in range(5):
    for j in range(units_between_stride):
        layer1 = resUnit(layer1,j + (i*units_between_stride))
    layer1 = slim.conv2d(layer1,64,[3,3],stride=[2,2],normalizer_fn=slim.batch_norm,scope='conv_s_'+str(i))
    
top = slim.conv2d(layer1,10,[3,3],normalizer_fn=slim.batch_norm,activation_fn=None,scope='conv_top')

output = slim.layers.softmax(slim.layers.flatten(top))

loss = tf.reduce_mean(-tf.reduce_sum(label_oh * tf.log(output) + 1e-10, reduction_indices=[1]))
trainer = tf.train.AdamOptimizer(learning_rate=0.001)
update = trainer.minimize(loss)

In [25]:
W = tf.Variable(tf.zeros([1024,10]))
b = tf.Variable(tf.zeros([10]))

In [4]:
correct_pred=tf.equal(tf.argmax(Y_test,1),tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [5]:
init = tf.global_variables_initializer()

In [6]:
sess = tf.InteractiveSession()

In [7]:
sess.run(init)

In [ ]:
train_batch_size = 64

In [12]:
for i in range(1000):
  [batch_xs,batch_ys] = mnist.train.next_batch(100)
  sess.run(trainer, feed_dict={input_layer: batch_xs, output: batch_ys})

ValueError: Cannot feed value of shape (100, 784) for Tensor 'input:0', which has shape '(?, 32, 32, 3)'